In [ ]:
%reload_ext autoreload
%autoreload 3

import matplotlib.pyplot as plt
from nilearn import datasets, image, plotting

import pandas as  pd
import matplotlib.pyplot as plt
import numpy as np
from nilearn import datasets, image, plotting
from nilearn.datasets import load_mni152_template
from nilearn.maskers.nifti_spheres_masker import _apply_mask_and_get_affinity
from nilearn.masking import intersect_masks
from src.acnets.parcellations.dosenbach import load_dosenbach2010_masker
from src.acnets.parcellations.friedman import load_fiedman2020_masker
from src.acnets.parcellations.gordon import \
    fetch_atlas as fetch_gordon2014_atlas
from src.acnets.parcellations import seitzman
from tqdm.auto import tqdm

tqdm.pandas()